Basic recommender system with TFIDF and cosine similarity
//

Some improvements:
Now it is **not case sensitive** (lowercase everything in the function), I also **removed punctuation** to make it easier to match with the input movie title.

//
Source of the whole tutorial: https://www.youtube.com/watch?v=ijtxuF_5kEU

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [22]:
import pandas as pd
import numpy as np
import string # for translator (remove punctuation for later)

In [29]:
path =  '/content/drive/My Drive/Colab Notebooks/02_Recommendersystem/TMDB5000'

df_movies = pd.read_csv(path+"/tmdb_5000_movies.csv")
df_credits= pd.read_csv(path+"/tmdb_5000_credits.csv")

In [4]:
df_movies.head()
df_credits.head()

,movie_id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."
1,285,Pirates of the Caribbean: At World's End,"[{""cast_id"": 4, ""character"": ""Captain Jack Spa...","[{""credit_id"": ""52fe4232c3a36847f800b579"", ""de..."
2,206647,Spectre,"[{""cast_id"": 1, ""character"": ""James Bond"", ""cr...","[{""credit_id"": ""54805967c3a36829b5002c41"", ""de..."
3,49026,The Dark Knight Rises,"[{""cast_id"": 2, ""character"": ""Bruce Wayne / Ba...","[{""credit_id"": ""52fe4781c3a36847f81398c3"", ""de..."
4,49529,John Carter,"[{""cast_id"": 5, ""character"": ""John Carter"", ""c...","[{""credit_id"": ""52fe479ac3a36847f813eaa3"", ""de..."


In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

In [8]:
df_movies.columns

Index(['budget', 'genres', 'homepage', 'id', 'keywords', 'original_language',
       'original_title', 'overview', 'popularity', 'production_companies',
       'production_countries', 'release_date', 'revenue', 'runtime',
       'spoken_languages', 'status', 'tagline', 'title', 'vote_average',
       'vote_count'],
      dtype='object')

In [48]:
# fillna
df_movies["overview"] =df_movies["overview"].fillna("")
df_movies["genres"] = df_movies["genres"].fillna("")
df_movies["keywords"] = df_movies["keywords"].fillna("")

#lowercase original title for the input to match with this
df_movies["original_title"] = df_movies["original_title"].str.lower()
# remove punctuation
translator = str.maketrans("", "", string.punctuation)
df_movies["original_title"] = df_movies["original_title"].str.translate(translator)


##########
tfidf = TfidfVectorizer(stop_words="english")

df_movies["combined_features"] = df_movies["overview"] + " " + df_movies["genres"] + " " + df_movies["keywords"]

tfidf_matrix = tfidf.fit_transform(df_movies["combined_features"])


######
# cosine similarity
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
indices = pd.Series(df_movies.index, index= df_movies["original_title"]).drop_duplicates()


#######
def get_recommendations(title, cosine_sim = cosine_sim, df=df_movies):
  title = title.lower()
  title = title.translate(translator)
  try:
    idx = indices[title]
  except:
    print(f"Movie with title '{title}' not found.")
    return None


  idx =indices[title] # get the index, since we are moving from the two dataframes by the index of each movie
  sim_scores = enumerate(cosine_sim[idx]) # check the movie with this index
              # We have used enumerate to get both the index and the score in a tuple
  sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True) # I want to get the highest score

  sim_scores = sim_scores[1:10] # get scores from 1 to 10, why not from 0?? Because it will be itself (same score in cos similarity)

  sim_index = [i[0] for i in sim_scores]
  #print(sim_index)

  print(df_movies["original_title"].iloc[sim_index])

In [49]:
get_recommendations("Pirates of the Caribbean The Curse of the Black Pearl")

12         pirates of the caribbean dead mans chest
17       pirates of the caribbean on stranger tides
1            pirates of the caribbean at worlds end
3820                                     the pirate
1331                                    nims island
848     the pirates in an adventure with scientists
305                                 treasure planet
340                                cutthroat island
2217                       everyone says i love you
Name: original_title, dtype: object
